# <font color='navy'> Tarea 6: TRS y CDS
<font color='navy'> Luis Fernando Márquez Bañuelos | 744489

## <font color='navy'> Total Return Swaps

In [1]:
import numpy as np

In [2]:
def TRS(n_bonds: np.array, coupons: np.array, initial_prices: np.array, final_prices: np.array, fixed_rate: float, t: int, pay_total_return: bool):

    initial_value = np.dot(n_bonds, initial_prices)
    final_value = np.dot(n_bonds, final_prices)
    cuopons_pay = np.sum(100 * n_bonds * coupons/t)
    total_return = final_value - initial_value + cuopons_pay
    fixed_rate_pay = initial_value * fixed_rate / t

    return fixed_rate_pay - total_return if pay_total_return else total_return - fixed_rate_pay

In [3]:
n_bonds = np.array([35_000, 45_000, 50_000])
coupons = np.array([0.06, 0.07, 0.08])
initial_prices = np.array([98.75, 101.5, 97.6])
final_prices = np.array([96.4, 97.3, 92.8])
fixed_rate = 0.04
t = 2
pay_total_return = True

In [4]:
TRS(n_bonds, coupons, initial_prices, final_prices, fixed_rate, t, pay_total_return)

306825.0

In [5]:
n_bonds = np.array([40_000, 50_000, 30_000])
coupons = np.array([0.0650, 0.0575, 0.0425])
initial_prices = np.array([106.83, 101.45, 98.36])
final_prices = np.array([104.27, 100.67, 97.24])
fixed_rate = 0.0645
t = 4
pay_total_return = False

In [6]:
TRS(n_bonds, coupons, initial_prices, final_prices, fixed_rate, t, pay_total_return)

-204531.0625

## <font color='navy'> Credit Default Swaps

Spread del CDS:

$$
S_0 = \frac{\text{Protection Leg}}{RPV01}
$$

donde Protection Leg:

$$
\text{Protection Leg} = \int_0^T \ Z(\tau) (1-R) dPD(\tau)
$$

y RPV01:

$$
RPV01 = \sum_{j=1}^N Z(tj) \Delta (tj - 1, tj, B) Q (tj)
$$

- **¿Qué es el RPV01? ¿Como se interpreta? ¿Que representan los elementos utilizados para calcularlos?**

El RPV01 es el valor presente esperado de 1 basis point (bp) pagado en la premium leg hasta default o vencimiento. La R es de "risky" ya que el RPV01 es el valor presente esperado de un flujo de primas incierto. (Pág 3)

El RPV01 no dice que tan sensible es el valor de la premium leg ante un cabo de 1bp en el spread. Un RPV01 indica más sensibilidad en la premium leg por cada basis point que se mueve el spread. (Pág 3-4)

$$
RPV01 = \sum_{j=1}^N Z(tj) \Delta (tj - 1, tj, B) Q (tj)
$$

donde:

- $Z(tj)$: Tasa de descuento de la fecha de valuación hasta la fecha de pago de la prima. (Tasa Libor en el paper revisado)
- $\Delta (tj-1,tj,B)$: Fracción de año (conteo de días) entre fechas de pago de primas.
- $Q(tj)$: Probabilidad de supervivencia de la entidad de referencia desde fecha de valuación hasta fecha de pago de la prima. Toma en cuenta para la valueción del riesgo que la entidad de referencia no llegue a la fecha de pago de la prima.

(Pág 6-7)


- **¿Qué es el Protection Leg? ¿Como se interpreta? ¿Que representan los elementos utilizados para calcularlos?**

Es el pago (1-R) o LGD descontado a valor presente en caso que se de un evento de crédito. (Pág 8)

En geenral se considera como el valor presente de la pérdida esperada (LGD) ajustada por su probabilidad de default. Ya que si la PD es más grande la prima es mayor o si el recovery rate es más chico la prima es mayor (visto en clase).

$$
\text{Protection Leg} = \int_0^T \ Z(\tau) (1-R) dPD(\tau)
$$

donde:

- $Z(\tau)$: factor de descuento al tiempo $\tau$.
- $(1-R)$: Lo que se le paga al protection buyer (LGD) donde $R$ es el recovery rate.
- $dPD(\tau)$: densidad de la probabilidad de supervivencia al tiempo $\tau$.

(Pág 9)

- **¿Qué representa el spread de un CDS de manera matemática?, es decir, como se interpreta el ratio utilizado para calcularlo y que significa.**

El spread de un CDS se calcula de la siguiente manera:

$$
S_0 = \frac{\text{Protection Leg}}{RPV01}
$$

Aqui se utiliza tanto el Protection Leg como el RPV01 definidos anteriormente.

La interpretación directa es que tanto representa el Protection Leg (LGD esperado a valor presente) del RPV01 (valor esperado presente de 1 bp). Se le considera el spread de equilibrio ya que es el que iguala el valor presente de la premium leg y del protection leg. Además nos proporciona una relación directa entre el spread de mercado y la probabilidad de supervivencia implícita en el mercado.

(Pág 11)

- **¿Qué otros elementos de un CDS es importante conocer y como se calculan?**

Uno de los elementos importantes a conocer son los **`Hazard Rates`** que refleja las probabilidades de que la entidad de referencia tenga en un evento de crédito en distintos puntos del tiempo.

Esta estrucutra de hazard rates se construye a través de un proceso iterativo el cual es el bootstrapping. Inicia con la fecha de vencimiento más cercana para calcular la probabilidad de supervivencia. Se calcula de la siguiente manera (asumiendo pagos timestrales) :

$$
\frac{S(t_v, t_v + 1Y)}{1- R} \ \sum_{n=3,6,9,12} \Delta(t_{n-3},t_n,B) Z(t_v,t_n)e^{-\lambda_{01}\tau_n} = \sum_{m=1}^{12} Z(t_v,t_m)(e^{-\lambda_{01}\tau_{m-1}}-e^{\lambda_{01}\tau_m})
$$

Esto se hace para $\lambda_{0,1}$ y la ecuación se resuelve con one-dimensional root-searching algorythm. El proceso se repite para todas las $\lambda$'s hasta llegar al vencimiento. Así obtienes las probabilidades de supervivencia en distintos puntos del tiempo, y te permite ver si el mercado percibe más riesgo a largo plazo o menos riesgo.

El cálculo de estas probabilidades es esencial para obtener PD, y calcular tanto el Protection Leg como el RPV01 que son los elementos necesarios para obtener el spread.

(Pág 13)

Otro concepto importante es el **`Market-to-Market Value (MTM)`** el cual se calcula de la siguiente manera (fecha de valuación es una fecha de pago de prima):

$$
MTM(t_V,t_N) = \pm [S(t_V,t_N) - S(t_0,t_N)] \times RPV01(t_V,t_N)
$$

Esto representa el valor presente de un CDS en el mercado. Es la ganacia o pérdida del titular del contrato en caso de cerrar su posicón. La fórmula utilizada se debe a que el PNL de un CDS no puede ser calculado igual que el de los bonos.

(Pág 3 y 15)

## <font color='navy'> Referencias Bibliográficas

<font color='black'>

O’Kane, D., & Turnbull, S. (2003). *Valuation of credit default swaps*. Quantitative Credit Research QCR Quarterly (2003-Q1/Q2). Recuperado el 30 de abril de 2025, de https://www.researchgate.net/publication/228872070_Valuation_of_credit_default_swaps

Páginas (3-15)